In [7]:
%load_ext dotenv
%dotenv
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}

def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

def fetch_web_page(url: str) -> str:
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

In [9]:
ddg_search = DuckDuckGoSearchResults()

web_fetch_tool = Tool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="Fetches the content of a web page"
)

prompt_template = "Summarize the following content: {content}"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

summarize_tool = Tool.from_function(
    func=llm_chain.run,
    name="Summarizer",
    description="Summarizes a web page"
)

tools = [ddg_search, web_fetch_tool, summarize_tool]

agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)



In [10]:
prompt = "I am want to travel to UK from Sweden for 7 days. I want my trip to be as sustainable and environmental friendly as possible. Give a plan for 7 days including transport and attractions to visit"

print(agent.run(prompt))



> Entering new AgentExecutor chain...
To create a sustainable and environmentally friendly travel plan, I need to consider modes of transportation and attractions that align with these principles.
Action: DuckDuckGo Results JSON
Action Input: "sustainable transportation options in UK"
Observation: [snippet: 5 UK Cities Leading the Way in Sustainable Transport As the world strives to combat climate change, the need for transportation systems that prioritize environmental sustainability has gained significant momentum., title: 5 UK Cities Leading the Way in Sustainable Transport, link: https://www.plugandplaytechcenter.com/resources/5-uk-cities-sustainable-transport/], [snippet: Over half of the UK public plan to increase their use of sustainable transport choices in 2023, as a way to cut spending and look after the planet, says FREE NOW's new research., title: Over half of UK public pledging to use more sustainable mobility in ..., link: https://www.intelligenttransport.com/transport-

In [11]:
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [12]:
result = agent.run(prompt)
print(result)



> Entering new PlanAndExecute chain...
steps=[Step(value='Research sustainable transportation options from Sweden to the UK, such as trains or buses, that have lower carbon emissions compared to flying.'), Step(value='Book your transportation tickets in advance to secure the best fares and ensure availability.'), Step(value='Choose eco-friendly accommodations in the UK that prioritize sustainability, such as hotels with green certifications or eco-lodges.'), Step(value='Create an itinerary for the 7 days, selecting attractions and activities that align with your sustainability goals. Consider visiting places known for their eco-friendly practices, such as national parks, organic farms, or renewable energy sites.'), Step(value='Use public transportation or walk/cycle whenever possible to explore the cities and attractions you plan to visit.'), Step(value='Research sustainable dining options in the UK, such as vegetarian or vegan restaurants, farm-to-table establishments, or eateries t